In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LayerNormalization, LeakyReLU, BatchNormalization
from tensorflow.keras.layers import LSTM
from keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Bidirectional

### Train Data

In [36]:
baseline_train_data = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"]:
    df = pd.concat([pd.read_csv(f"LSTM_data/{coin}usdt/training_output.csv"),\
                   pd.read_csv(f"LSTM_data/{coin}usdt/validation_output.csv")])
    baseline_train_data.append(df)

baseline_train_data[0]

,Open time,LogReturns,LogLow/O,LogHigh/O
0,1609545600000,-0.019493,-0.027628,0.003820
1,1609549200000,0.010832,-0.009243,0.014395
2,1609552800000,0.004025,-0.002249,0.006430
3,1609556400000,0.004180,-0.000804,0.010958
4,1609560000000,-0.004065,-0.010743,0.000914
...,...,...,...,...
2154,1711908000000,0.000775,-0.002950,0.005720
2155,1711911600000,0.002630,-0.004658,0.004945
2156,1711915200000,0.002623,-0.001701,0.004009
2157,1711918800000,-0.004478,-0.006493,0.000000


In [37]:
df = baseline_train_data[0]
df[np.isinf(df["LogHigh/O"])]

,Open time,LogReturns,LogLow/O,LogHigh/O


In [56]:
scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.reshape(-1, 3)).reshape(-1, 24, 3)
# y_train = scaler.fit_transform(y_train)

def generate_data(dfs):
    X, y = [], []
    for df in dfs:
        values = df[["LogReturns", "LogLow/O", "LogHigh/O"]].values
        for i in range(24, len(values)):
            X.append(values[i-24:i])
            y.append(values[i][0])

    return scaler.fit_transform(np.array(X).reshape(-1, 3)).reshape(-1, 24, 3),\
           scaler.fit_transform(np.array(y).reshape(-1, 1))

X_train, y_train = generate_data(baseline_train_data)

In [69]:
print(X_train.shape)
print(y_train.shape)

(280197, 24, 3)
(280197, 1)


In [ ]:
baseline_model = Sequential([
    Bidirectional(LSTM(32, return_sequences=True), input_shape=(24, 3)),
    BatchNormalization(),
    Dropout(0.5),

    Bidirectional(LSTM(32, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.5),

    Flatten(),

    # Dense(16, activation='relu'),
    # BatchNormalization(),
    # Dropout(0.5),
    
    Dense(1)
])

baseline_model.compile(optimizer='adam', loss='mean_squared_error')
baseline_model.summary()


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_35                │ (None, 24, 64)         │         9,216 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_63          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_72 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_36                │ (None, 24, 64)         │        24,832 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_64          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_73 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 1)              │         1,537 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,097 (141.00 KB)

 Trainable params: 35,841 (140.00 KB)

 Non-trainable params: 256 (1.00 KB)

In [84]:
baseline_model.fit(X_train, y_train, epochs=2, validation_split=0.1, verbose=1)

Epoch 1/2
7881/7881 ━━━━━━━━━━━━━━━━━━━━ 198s 25ms/step - loss: 1.3848 - val_loss: 0.1673
Epoch 2/2
7881/7881 ━━━━━━━━━━━━━━━━━━━━ 186s 24ms/step - loss: 1.2320 - val_loss: 0.1683


In [78]:
loss = baseline_model.evaluate(X_train, y_train, verbose=1)
print(f"Train Loss (MSE): {loss:.6f}")

# Test against baseline of predicting 0
baseline_loss = np.mean(y_train ** 2)
print(f"0 Loss: {baseline_loss:.6f}")

8757/8757 ━━━━━━━━━━━━━━━━━━━━ 55s 6ms/step - loss: 0.6773
Train Loss (MSE): 1.000717
0 Loss: 1.000000


### Test Data

In [71]:
baseline_test_data = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"] + ["bonk", "floki", "pepe", "wif"]:
    baseline_test_data.append(pd.read_csv(f"LSTM_data/{coin}usdt/testing_output.csv"))

baseline_test_data[0]

,Open time,LogReturns,LogLow/O,LogHigh/O
0,1712016000000,-0.006622,-0.009870,0.000483
1,1712019600000,0.001943,-0.007317,0.004203
2,1712023200000,-0.032046,-0.054175,0.000323
3,1712026800000,-0.005190,-0.007880,0.005994
4,1712030400000,-0.000168,-0.002185,0.007026
...,...,...,...,...
5106,1730397600000,-0.003203,-0.011108,0.000000
5107,1730401200000,-0.005850,-0.006728,0.002620
5108,1730404800000,0.004974,-0.002055,0.007596
5109,1730408400000,0.001167,-0.003215,0.004077


In [72]:
X_test, y_test = generate_data(baseline_test_data)
print(X_test.shape)
print(y_test.shape)

(69961, 24, 3)
(69961, 1)


In [79]:
loss = baseline_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss (MSE): {loss:.6f}")

# Test against baseline of predicting 0
baseline_loss = np.mean(y_test ** 2)
print(f"0 Loss: {baseline_loss:.6f}")

2187/2187 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.6704
Test Loss (MSE): 1.000930
0 Loss: 1.000000
